In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e7/sample_submission.csv
/kaggle/input/playground-series-s5e7/train.csv
/kaggle/input/playground-series-s5e7/test.csv


# Setup

In [2]:
import random, os, gc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix, classification_report

# Load Data

In [3]:
train = pd.read_csv("/kaggle/input/playground-series-s5e7/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s5e7/test.csv")

# EDA

In [4]:
train.head(10)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert
5,5,2.0,No,8.0,5.0,No,NaN,3.0,Extrovert
6,6,1.0,No,8.0,NaN,No,NaN,4.0,Extrovert
7,7,2.0,No,8.0,3.0,No,4.0,5.0,Extrovert
8,8,4.0,Yes,2.0,1.0,NaN,0.0,2.0,Introvert
9,9,1.0,No,8.0,6.0,No,14.0,9.0,Extrovert


In [5]:
train.tail(10)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
18514,18514,6.0,No,5.0,3.0,No,10.0,4.0,Extrovert
18515,18515,NaN,No,9.0,4.0,No,5.0,8.0,Extrovert
18516,18516,NaN,No,6.0,5.0,No,9.0,4.0,Extrovert
18517,18517,3.0,No,NaN,4.0,No,7.0,6.0,Extrovert
18518,18518,3.0,NaN,8.0,3.0,No,5.0,8.0,Extrovert
18519,18519,3.0,No,7.0,3.0,No,9.0,7.0,Extrovert
18520,18520,1.0,NaN,6.0,7.0,No,6.0,5.0,Extrovert
18521,18521,7.0,Yes,1.0,1.0,Yes,1.0,NaN,Introvert
18522,18522,NaN,Yes,1.0,0.0,Yes,5.0,2.0,Introvert
18523,18523,1.0,No,8.0,6.0,No,4.0,7.0,Extrovert


In [6]:
train.describe()

,id,Time_spent_Alone,Social_event_attendance,Going_outside,Friends_circle_size,Post_frequency
count,18524.000000,17334.000000,17344.000000,17058.000000,17470.000000,17260.000000
mean,9261.500000,3.137764,5.265106,4.044319,7.996737,4.982097
std,5347.562529,3.003786,2.753359,2.062580,4.223484,2.879139
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4630.750000,1.000000,3.000000,3.000000,5.000000,3.000000
50%,9261.500000,2.000000,5.000000,4.000000,8.000000,5.000000
75%,13892.250000,4.000000,8.000000,6.000000,12.000000,7.000000
max,18523.000000,11.000000,10.000000,7.000000,15.000000,10.000000


In [7]:
train.isnull().sum() # Check for missing values in train

id                              0
Time_spent_Alone             1190
Stage_fear                   1893
Social_event_attendance      1180
Going_outside                1466
Drained_after_socializing    1149
Friends_circle_size          1054
Post_frequency               1264
Personality                     0
dtype: int64

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18524 entries, 0 to 18523
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         18524 non-null  int64  
 1   Time_spent_Alone           17334 non-null  float64
 2   Stage_fear                 16631 non-null  object 
 3   Social_event_attendance    17344 non-null  float64
 4   Going_outside              17058 non-null  float64
 5   Drained_after_socializing  17375 non-null  object 
 6   Friends_circle_size        17470 non-null  float64
 7   Post_frequency             17260 non-null  float64
 8   Personality                18524 non-null  object 
dtypes: float64(5), int64(1), object(3)
memory usage: 1.3+ MB


In [9]:
test.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
0,18524,3.0,No,7.0,4.0,No,6.0,NaN
1,18525,NaN,Yes,0.0,0.0,Yes,5.0,1.0
2,18526,3.0,No,5.0,6.0,No,15.0,9.0
3,18527,3.0,No,4.0,4.0,No,5.0,6.0
4,18528,9.0,Yes,1.0,2.0,Yes,1.0,1.0


In [10]:
test.isnull().sum()

id                             0
Time_spent_Alone             425
Stage_fear                   598
Social_event_attendance      397
Going_outside                466
Drained_after_socializing    432
Friends_circle_size          350
Post_frequency               408
dtype: int64

In [11]:
test.describe()

,id,Time_spent_Alone,Social_event_attendance,Going_outside,Friends_circle_size,Post_frequency
count,6175.000000,5750.000000,5778.000000,5709.000000,5825.000000,5767.000000
mean,21611.000000,3.116870,5.287989,4.037835,8.008412,5.028958
std,1782.713288,2.985658,2.758052,2.045207,4.192701,2.867285
min,18524.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20067.500000,1.000000,3.000000,3.000000,5.000000,3.000000
50%,21611.000000,2.000000,5.000000,4.000000,8.000000,5.000000
75%,23154.500000,4.000000,8.000000,6.000000,12.000000,7.000000
max,24698.000000,11.000000,10.000000,7.000000,15.000000,10.000000


## Replacing Mising Values

In [12]:
TARGET   = "Personality"     
ID_COL   = "id"

# Drop the target (and id) from the feature frame
feat_train = train.drop(columns=[TARGET, ID_COL])
feat_test  = test.drop(columns=[ID_COL])    

In [13]:
# Split columns by dtype
num_cols = feat_train.select_dtypes(include='number').columns
cat_cols = feat_train.select_dtypes(include='object').columns

In [14]:
# Fill numeric with median
feat_train[num_cols] = feat_train[num_cols].fillna(train[num_cols].median())
feat_test[num_cols] = feat_test[num_cols].fillna(train[num_cols].median())

In [15]:
# Fill categoricals with 'Missing'
feat_train[cat_cols] = feat_train[cat_cols].fillna("Missing")
feat_test[cat_cols] = feat_test[cat_cols].fillna("Missing")

In [16]:
pd.concat([feat_train, feat_test]).isna().sum().sum()

0

# K‑fold cross‑validation to pick iterations & score

In [17]:
from catboost import CatBoostClassifier, Pool


SEED      = 42
N_FOLDS   = 5

# ========= 1.  split label & features =========
y = train[TARGET].map({"Introvert":0, "Extrovert":1})  # convert to 0/1
X = feat_train.copy()

cat_cols = X.select_dtypes(include="object").columns.tolist()

In [18]:
oof_pred   = np.zeros(len(feat_train))
fold_scores, best_iters = [], []

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

for fold, (tr_idx, val_idx) in enumerate(skf.split(feat_train, y)):
    print(f"\n🟢  Fold {fold+1}/{N_FOLDS}")
    X_tr, X_val = feat_train.iloc[tr_idx], feat_train.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    tr_pool  = Pool(X_tr, y_tr, cat_features=cat_cols)
    val_pool = Pool(X_val, y_val, cat_features=cat_cols)

    model = CatBoostClassifier(
        iterations=5000,                 # upper bound; we'll early‑stop
        learning_rate=0.05,
        depth=6,
        l2_leaf_reg=3,
        loss_function="Logloss",
        eval_metric="Logloss",
        random_seed=SEED,
        verbose=False,
        early_stopping_rounds=300
    )
    model.fit(tr_pool, eval_set=val_pool, use_best_model=True)
    best_iters.append(model.get_best_iteration())

    val_pred = model.predict_proba(val_pool)[:, 1]
    oof_pred[val_idx] = val_pred
    score = log_loss(y_val, val_pred)
    fold_scores.append(score)
    print(f"    Fold log‑loss: {score:.5f}  |  best_iter: {model.get_best_iteration()}")

print("\n📊  CV mean log‑loss =", np.mean(fold_scores), "+/-", np.std(fold_scores))
BEST_ITERS = int(np.round(np.mean(best_iters)))
print("Chosen global iteration count:", BEST_ITERS)


🟢  Fold 1/5
    Fold log‑loss: 0.12589  |  best_iter: 138

🟢  Fold 2/5
    Fold log‑loss: 0.13563  |  best_iter: 167

🟢  Fold 3/5
    Fold log‑loss: 0.14105  |  best_iter: 88

🟢  Fold 4/5
    Fold log‑loss: 0.12607  |  best_iter: 158

🟢  Fold 5/5
    Fold log‑loss: 0.12107  |  best_iter: 80

📊  CV mean log‑loss = 0.12994119054847944 +/- 0.007292196983726723
Chosen global iteration count: 126


# Train final model on the **full** training dataset

In [19]:
full_pool = Pool(feat_train, y, cat_features=cat_cols)
final_model = CatBoostClassifier(
    iterations=BEST_ITERS,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3,
    loss_function="Logloss",
    eval_metric="Logloss",
    random_seed=SEED,
    verbose=False
)
final_model.fit(full_pool)


# Predict test set and create submission

In [20]:
test_pool = Pool(feat_test, cat_features=cat_cols)
test_probs = final_model.predict_proba(test_pool)[:, 1]
test_labels = np.where(test_probs >= 0.5, "Extrovert", "Introvert")

submission = pd.DataFrame({
    ID_COL: test[ID_COL],
    TARGET: test_labels
})
submission.to_csv("submission.csv", index=False)
print(submission.head())

      id Personality
0  18524   Extrovert
1  18525   Introvert
2  18526   Extrovert
3  18527   Extrovert
4  18528   Introvert
